# Fine-Tuning Ultralytics Models: A Comprehensive Course

## Course Overview

Welcome to this comprehensive guide on fine-tuning state-of-the-art object detection models from Ultralytics. In this course, you will learn how to:

- Fine-tune **YOLOv5, YOLOv8, YOLOv9, YOLOv10, YOLOv11**, and **RT-DETR**
- Train models on custom datasets
- Run inference and evaluate model performance
- Modify and optimize hyperparameters
- Apply advanced training techniques

---

## Table of Contents

1. [Introduction to Fine-Tuning](#1-introduction)
2. [Prerequisites and Setup](#2-setup)
3. [Dataset Preparation](#3-dataset)
4. [YOLOv5 Fine-Tuning](#4-yolov5)
5. [YOLOv8 Fine-Tuning](#5-yolov8)
6. [YOLOv9 Fine-Tuning](#6-yolov9)
7. [YOLOv10 Fine-Tuning](#7-yolov10)
8. [YOLOv11 Fine-Tuning](#8-yolov11)
9. [RT-DETR Fine-Tuning](#9-rtdetr)
10. [Hyperparameter Tuning Guide](#10-hyperparameters)
11. [Advanced Techniques](#11-advanced)
12. [Comparison and Best Practices](#12-comparison)

---

## 1. Introduction to Fine-Tuning <a id="1-introduction"></a>

### What is Fine-Tuning?

Fine-tuning is the process of taking a pre-trained model and adapting it to a specific task or dataset. Instead of training a model from scratch, we leverage the knowledge the model has already learned from large datasets.

### Why Fine-Tune?

- **Faster Training**: Pre-trained models already understand basic features
- **Better Performance**: Especially with limited data
- **Less Data Required**: Can achieve good results with smaller datasets
- **Resource Efficient**: Requires less computational power than training from scratch

### Model Overview

| Model | Release | Key Features | Speed | Accuracy |
|-------|---------|--------------|-------|----------|
| YOLOv5 | 2020 | Mature, stable, excellent documentation | Fast | Good |
| YOLOv8 | 2023 | Unified API, multiple tasks support | Very Fast | Better |
| YOLOv9 | 2024 | Programmable gradient information | Fast | Excellent |
| YOLOv10 | 2024 | Real-time end-to-end detection | Very Fast | Excellent |
| YOLOv11 | 2024 | Latest, state-of-the-art performance | Very Fast | Best |
| RT-DETR | 2023 | Transformer-based, no NMS needed | Fast | Excellent |

---

## 2. Prerequisites and Setup <a id="2-setup"></a>

### Install Required Libraries

In [ ]:
# Install Ultralytics package (includes YOLOv8, YOLOv9, YOLOv10, YOLOv11, RT-DETR)
!pip install -q ultralytics

# Install YOLOv5 (separate repository)
!pip install -q yolov5

# Additional useful libraries
!pip install -q roboflow  # For dataset management
!pip install -q pillow numpy matplotlib opencv-python

In [ ]:
# Import necessary libraries
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from pathlib import Path

# Check for GPU availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Check Ultralytics version
import ultralytics
print(f"Ultralytics version: {ultralytics.__version__}")

# Verify installation
ultralytics.checks()

---

## 3. Dataset Preparation <a id="3-dataset"></a>

### Dataset Format

All Ultralytics models use the YOLO format for datasets:

```
dataset/
├── images/
│   ├── train/
│   │   ├── image1.jpg
│   │   ├── image2.jpg
│   │   └── ...
│   └── val/
│       ├── image1.jpg
│       └── ...
└── labels/
    ├── train/
    │   ├── image1.txt
    │   ├── image2.txt
    │   └── ...
    └── val/
        ├── image1.txt
        └── ...
```

### Label Format

Each `.txt` file contains one line per object:
```
class_id center_x center_y width height
```

Where all coordinates are normalized (0-1):
- `class_id`: Integer starting from 0
- `center_x`, `center_y`: Center coordinates of bounding box
- `width`, `height`: Width and height of bounding box

Example:
```
0 0.5 0.5 0.3 0.4
1 0.2 0.3 0.15 0.2
```

### Create a Dataset Configuration File

Create a YAML file describing your dataset:

In [ ]:
# Example: Create a dataset.yaml file
dataset_yaml = """
# Dataset configuration
path: /path/to/dataset  # Root directory
train: images/train  # Training images (relative to 'path')
val: images/val  # Validation images (relative to 'path')

# Number of classes
nc: 2

# Class names
names: ['class1', 'class2']
"""

# Save the configuration
with open('dataset.yaml', 'w') as f:
    f.write(dataset_yaml)

print("Dataset configuration saved to 'dataset.yaml'")

### Download Sample Dataset (COCO8 for demonstration)

In [ ]:
# https://universe.roboflow.com/timilehin-morighanfen-v8icx/vehicles-5rakd/dataset/1/download

---

## 4. YOLOv5 Fine-Tuning <a id="4-yolov5"></a>

YOLOv5 is a mature and widely-used object detection model with excellent documentation and community support.

### 4.1 Training YOLOv5

In [ ]:
# Clone YOLOv5 repository (if not already done)
if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5
    !pip install -q -r yolov5/requirements.txt

In [ ]:
# Training YOLOv5 using the command line approach
!python yolov5/train.py \
    --img 640 \
    --batch 16 \
    --epochs 50 \
    --data {dataset_path} \
    --weights yolov5s.pt \
    --device 0 \
    --project runs/yolov5 \
    --name custom_training \
    --cache

### 4.2 YOLOv5 Training with Python API

In [ ]:
# Alternative: Using Python API
import torch

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Training is still typically done via the train.py script
# But you can use the yolov5 package for inference

### 4.3 YOLOv5 Inference

In [ ]:
# Load trained model for inference
model = torch.hub.load('ultralytics/yolov5', 'custom', 
                       path='runs/yolov5/custom_training/weights/best.pt')

# Run inference on an image
img_path = 'path/to/test/image.jpg'
results = model(img_path)

# Display results
results.show()

# Print detections
results.print()

# Get pandas dataframe with results
df = results.pandas().xyxy[0]
print(df)

### 4.4 YOLOv5 Hyperparameter Tuning

In [ ]:
# YOLOv5 training with custom hyperparameters
!python yolov5/train.py \
    --img 640 \
    --batch 16 \
    --epochs 100 \
    --data {dataset_path} \
    --weights yolov5s.pt \
    --device 0 \
    --project runs/yolov5 \
    --name custom_hyperparams \
    --hyp data/hyps/hyp.scratch-low.yaml \
    --optimizer Adam \
    --lr0 0.001 \
    --lrf 0.01 \
    --momentum 0.937 \
    --weight-decay 0.0005 \
    --warmup-epochs 3.0 \
    --cos-lr

### 4.5 YOLOv5 Evaluation

In [ ]:
# Validate the trained model
!python yolov5/val.py \
    --weights runs/yolov5/custom_training/weights/best.pt \
    --data {dataset_path} \
    --img 640 \
    --batch 16 \
    --device 0

---

## 5. YOLOv8 Fine-Tuning <a id="5-yolov8"></a>

YOLOv8 introduced a unified API that makes training and inference much simpler. It supports multiple tasks: detection, segmentation, classification, and pose estimation.

### 5.1 Training YOLOv8

In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8 model
# Available models: yolov8n, yolov8s, yolov8m, yolov8l, yolov8x
# n=nano (smallest), s=small, m=medium, l=large, x=xlarge (biggest)
model = YOLO('yolov8n.pt')  # Load the nano model

# Train the model
results = model.train(
    data=dataset_path,
    epochs=100,
    imgsz=640,
    batch=16,
    device=0,  # GPU device, or 'cpu'
    project='runs/yolov8',
    name='custom_training',
    patience=50,
    save=True,
    cache=True,
    pretrained=True,
    verbose=True
)

print("Training completed!")

### 5.2 YOLOv8 Training with Custom Hyperparameters

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8s.pt')

# Training with detailed hyperparameters
results = model.train(
    # Dataset
    data=dataset_path,
    
    # Training duration
    epochs=100,
    time=None,  # Maximum training time in hours
    patience=50,
    
    # Batch and image size
    batch=16,
    imgsz=640,
    
    # Learning rate settings
    lr0=0.01,  # Initial learning rate
    lrf=0.01,  # Final learning rate (lr0 * lrf)
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    
    # Optimizer
    optimizer='SGD',  # Options: 'SGD', 'Adam', 'AdamW', 'NAdam', 'RAdam', 'RMSProp'
    
    # Loss weights
    box=7.5,  # Box loss weight
    cls=0.5,  # Classification loss weight
    dfl=1.5,  # Distribution focal loss weight
    
    # Data augmentation
    hsv_h=0.015,  # Hue augmentation
    hsv_s=0.7,    # Saturation augmentation
    hsv_v=0.4,    # Value augmentation
    degrees=0.0,   # Rotation
    translate=0.1, # Translation
    scale=0.5,     # Scale
    shear=0.0,     # Shear
    perspective=0.0,  # Perspective
    flipud=0.0,    # Flip up-down
    fliplr=0.5,    # Flip left-right
    mosaic=1.0,    # Mosaic augmentation
    mixup=0.0,     # Mixup augmentation
    copy_paste=0.0,  # Copy-paste augmentation
    
    # Advanced settings
    cos_lr=False,  # Use cosine learning rate scheduler
    close_mosaic=10,  # Disable mosaic in last N epochs
    amp=True,  # Automatic Mixed Precision
    fraction=1.0,  # Dataset fraction to use
    
    # Device and workers
    device=0,
    workers=8,
    
    # Project settings
    project='runs/yolov8',
    name='hyperparameter_tuning',
    exist_ok=False,
    
    # Saving and validation
    save=True,
    save_period=-1,  # Save checkpoint every N epochs (-1 to disable)
    val=True,
    plots=True,
    
    # Other
    cache=True,
    rect=False,  # Rectangular training
    resume=False,  # Resume training
    seed=0,
    deterministic=True,
    verbose=True
)

### 5.3 YOLOv8 Automatic Hyperparameter Tuning

In [ ]:
from ultralytics import YOLO

# Load model
model = YOLO('yolov8n.pt')

# Tune hyperparameters using genetic algorithms
# This will automatically search for the best hyperparameters
results = model.tune(
    data=dataset_path,
    epochs=30,  # Epochs per tuning iteration
    iterations=300,  # Number of tuning iterations
    optimizer='AdamW',
    plots=True,
    save=True,
    val=True
)

print("Hyperparameter tuning completed!")
print(f"Best hyperparameters: {results}")

### 5.4 YOLOv8 Inference

In [ ]:
from ultralytics import YOLO
from PIL import Image

# Load trained model
model = YOLO('runs/yolov8/custom_training/weights/best.pt')

# Single image inference
results = model('path/to/image.jpg')

# Display results
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    im.show()  # show image
    
# Get predictions
boxes = results[0].boxes  # Boxes object
print(f"Detected {len(boxes)} objects")
print(f"Class IDs: {boxes.cls}")
print(f"Confidences: {boxes.conf}")
print(f"Bounding boxes: {boxes.xyxy}")

In [ ]:
# Inference with custom parameters
results = model.predict(
    source='path/to/image.jpg',
    conf=0.25,  # Confidence threshold
    iou=0.45,   # NMS IoU threshold
    imgsz=640,  # Image size
    device=0,   # GPU device
    save=True,  # Save results
    save_txt=True,  # Save results as .txt
    save_conf=True,  # Save confidences in .txt
    classes=[0, 1],  # Filter by class (optional)
    agnostic_nms=False,  # Class-agnostic NMS
    max_det=300,  # Maximum detections per image
    project='runs/detect',
    name='exp',
    show=True,  # Show results
    verbose=True
)

# Batch inference (multiple images, video, directory)
results = model.predict(
    source='path/to/images/',  # Can be directory, video, URL, etc.
    stream=True  # Use streaming for memory efficiency with large datasets
)

for result in results:
    print(f"Processing: {result.path}")
    boxes = result.boxes
    print(f"Found {len(boxes)} objects")

### 5.5 YOLOv8 Validation

In [ ]:
from ultralytics import YOLO

# Load model
model = YOLO('runs/yolov8/custom_training/weights/best.pt')

# Validate model
metrics = model.val(
    data=dataset_path,
    imgsz=640,
    batch=16,
    conf=0.25,
    iou=0.6,
    device=0
)

# Print metrics
print(f"mAP50: {metrics.box.map50}")
print(f"mAP50-95: {metrics.box.map}")
print(f"Precision: {metrics.box.mp}")
print(f"Recall: {metrics.box.mr}")

### 5.6 YOLOv8 Export (for deployment)

In [ ]:
from ultralytics import YOLO

# Load model
model = YOLO('runs/yolov8/custom_training/weights/best.pt')

# Export to different formats
model.export(format='onnx')  # ONNX
model.export(format='torchscript')  # TorchScript
model.export(format='coreml')  # CoreML
model.export(format='tflite')  # TensorFlow Lite
model.export(format='engine')  # TensorRT

print("Model exported successfully!")

---

## 6. YOLOv9 Fine-Tuning <a id="6-yolov9"></a>

YOLOv9 introduced Programmable Gradient Information (PGI) and Generalized Efficient Layer Aggregation Network (GELAN), improving accuracy without increasing computational cost.

### 6.1 Training YOLOv9

In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLOv9 model
# Available models: yolov9t, yolov9s, yolov9m, yolov9c, yolov9e
model = YOLO('yolov9c.pt')  # Load YOLOv9c

# Train the model
results = model.train(
    data=dataset_path,
    epochs=100,
    imgsz=640,
    batch=16,
    device=0,
    project='runs/yolov9',
    name='custom_training',
    optimizer='SGD',
    lr0=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    cos_lr=True,
    patience=50,
    cache=True,
    verbose=True
)

print("YOLOv9 training completed!")

### 6.2 YOLOv9 Hyperparameter Tuning

In [ ]:
from ultralytics import YOLO

# YOLOv9 supports the same tuning API as YOLOv8
model = YOLO('yolov9c.pt')

# Automatic hyperparameter tuning
results = model.tune(
    data=dataset_path,
    epochs=30,
    iterations=300,
    optimizer='AdamW',
    plots=True,
    save=True,
    val=True
)

print(f"Best hyperparameters: {results}")

### 6.3 YOLOv9 Inference

In [ ]:
from ultralytics import YOLO

# Load trained YOLOv9 model
model = YOLO('runs/yolov9/custom_training/weights/best.pt')

# Run inference (same API as YOLOv8)
results = model.predict(
    source='path/to/image.jpg',
    conf=0.25,
    iou=0.45,
    save=True,
    show=True
)

# Process results
for r in results:
    boxes = r.boxes
    print(f"Detected {len(boxes)} objects")
    print(f"Classes: {boxes.cls}")
    print(f"Confidences: {boxes.conf}")

---

## 7. YOLOv10 Fine-Tuning <a id="7-yolov10"></a>

YOLOv10 introduced NMS-free training and efficient architecture design, achieving real-time performance with state-of-the-art accuracy.

### 7.1 Training YOLOv10

In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLOv10 model
# Available models: yolov10n, yolov10s, yolov10m, yolov10b, yolov10l, yolov10x
model = YOLO('yolov10n.pt')  # Load YOLOv10-nano

# Train the model
results = model.train(
    data=dataset_path,
    epochs=100,
    imgsz=640,
    batch=16,
    device=0,
    project='runs/yolov10',
    name='custom_training',
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    momentum=0.9,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    cos_lr=True,
    close_mosaic=10,
    amp=True,
    patience=50,
    cache=True,
    verbose=True
)

print("YOLOv10 training completed!")

### 7.2 YOLOv10 with Advanced Training Settings

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov10s.pt')

# Training with multi-scale and advanced augmentation
results = model.train(
    data=dataset_path,
    epochs=150,
    imgsz=640,
    batch=16,
    
    # Multi-scale training
    multi_scale=True,
    
    # Advanced augmentation
    mosaic=1.0,
    mixup=0.15,
    copy_paste=0.1,
    degrees=10.0,
    translate=0.2,
    scale=0.9,
    fliplr=0.5,
    flipud=0.1,
    
    # Learning rate schedule
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.001,
    cos_lr=True,
    
    # Other settings
    device=0,
    project='runs/yolov10',
    name='advanced_training',
    patience=75,
    val=True,
    plots=True
)

### 7.3 YOLOv10 Inference

In [ ]:
from ultralytics import YOLO

# Load trained YOLOv10 model
model = YOLO('runs/yolov10/custom_training/weights/best.pt')

# YOLOv10 doesn't require NMS, making inference faster
results = model.predict(
    source='path/to/image.jpg',
    conf=0.25,
    save=True,
    show=True,
    verbose=True
)

# Process results
for r in results:
    boxes = r.boxes
    print(f"Detected {len(boxes)} objects")
    for i, box in enumerate(boxes):
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        print(f"Object {i+1}: Class={cls}, Confidence={conf:.2f}")

---

## 8. YOLOv11 Fine-Tuning <a id="8-yolov11"></a>

YOLOv11 is the latest and most advanced YOLO version, offering state-of-the-art performance across all tasks with improved architecture and training techniques.

### 8.1 Training YOLOv11

In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLOv11 model
# Available models: yolo11n, yolo11s, yolo11m, yolo11l, yolo11x
model = YOLO('yolo11n.pt')  # or 'yolo11s.pt', 'yolo11m.pt', etc.

# Train the model
results = model.train(
    data=dataset_path,
    epochs=100,
    imgsz=640,
    batch=16,
    device=0,
    project='runs/yolo11',
    name='custom_training',
    optimizer='auto',  # Auto-select best optimizer
    lr0=0.01,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    cos_lr=True,
    patience=50,
    cache=True,
    amp=True,
    verbose=True
)

print("YOLOv11 training completed!")

### 8.2 YOLOv11 with PyTorch 2.x Compilation

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11s.pt')

# Use PyTorch 2.x torch.compile for faster training (requires PyTorch >= 2.0)
results = model.train(
    data=dataset_path,
    epochs=100,
    imgsz=640,
    batch=16,
    device=0,
    
    # Enable compilation for speed boost
    compile=True,  # or 'default', 'reduce-overhead', 'max-autotune'
    
    # Other settings
    optimizer='AdamW',
    lr0=0.001,
    cos_lr=True,
    amp=True,
    project='runs/yolo11',
    name='compiled_training',
    patience=50
)

print("Compiled training completed!")

### 8.3 YOLOv11 Transfer Learning with Layer Freezing

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11m.pt')

# Freeze the first 10 layers (backbone)
# This speeds up training and can improve performance on small datasets
results = model.train(
    data=dataset_path,
    epochs=50,
    imgsz=640,
    batch=16,
    device=0,
    
    # Freeze backbone layers
    freeze=10,  # Freeze first 10 layers, or use [0,1,2,3] for specific layers
    
    # Use higher learning rate since we're only training head
    lr0=0.01,
    
    project='runs/yolo11',
    name='transfer_learning',
    patience=30
)

print("Transfer learning completed!")

### 8.4 YOLOv11 Inference

In [ ]:
from ultralytics import YOLO
import cv2
from PIL import Image

# Load trained YOLOv11 model
model = YOLO('runs/yolo11/custom_training/weights/best.pt')

# Single image inference
results = model.predict(
    source='path/to/image.jpg',
    conf=0.25,
    iou=0.45,
    save=True,
    show=True
)

# Video inference
results = model.predict(
    source='path/to/video.mp4',
    conf=0.25,
    save=True,
    stream=True  # Stream for memory efficiency
)

for r in results:
    boxes = r.boxes
    print(f"Frame: {r.path}, Objects: {len(boxes)}")

### 8.5 YOLOv11 Real-time Webcam Inference

In [ ]:
from ultralytics import YOLO
import cv2

# Load model
model = YOLO('runs/yolo11/custom_training/weights/best.pt')

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Run inference
    results = model.predict(frame, conf=0.25, verbose=False)
    
    # Visualize results
    annotated_frame = results[0].plot()
    
    # Display
    cv2.imshow('YOLOv11 Inference', annotated_frame)
    
    # Break on 'q' press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

---

## 9. RT-DETR Fine-Tuning <a id="9-rtdetr"></a>

RT-DETR (Real-Time DEtection TRansformer) is a transformer-based detector that doesn't require Non-Maximum Suppression (NMS), making it more efficient for deployment.

### 9.1 Training RT-DETR

In [ ]:
from ultralytics import RTDETR

# Load a pre-trained RT-DETR model
# Available models: rtdetr-l, rtdetr-x
model = RTDETR('rtdetr-l.pt')

# Train the model
results = model.train(
    data=dataset_path,
    epochs=100,
    imgsz=640,
    batch=8,  # RT-DETR uses more memory, may need smaller batch
    device=0,
    project='runs/rtdetr',
    name='custom_training',
    optimizer='AdamW',
    lr0=0.0001,  # Lower learning rate for transformers
    lrf=0.0001,
    weight_decay=0.0001,
    warmup_epochs=5.0,  # Longer warmup for transformers
    patience=50,
    cache=False,  # May need to disable cache due to memory
    amp=False,  # RT-DETR can have issues with AMP
    verbose=True
)

print("RT-DETR training completed!")

### 9.2 RT-DETR with Custom Training Settings

In [ ]:
from ultralytics import RTDETR

model = RTDETR('rtdetr-l.pt')

# RT-DETR specific training configuration
results = model.train(
    data=dataset_path,
    epochs=100,
    imgsz=640,
    batch=8,
    
    # Optimizer settings (important for transformers)
    optimizer='AdamW',
    lr0=0.0001,
    lrf=0.00001,
    weight_decay=0.0001,
    
    # Warmup (critical for transformer training)
    warmup_epochs=5.0,
    warmup_momentum=0.5,
    warmup_bias_lr=0.0001,
    
    # Loss weights (may need adjustment for RT-DETR)
    box=5.0,
    cls=0.5,
    
    # Data augmentation (transformers benefit from strong augmentation)
    mosaic=1.0,
    mixup=0.1,
    copy_paste=0.1,
    degrees=10.0,
    translate=0.2,
    scale=0.9,
    
    # Other settings
    device=0,
    project='runs/rtdetr',
    name='optimized_training',
    patience=50,
    amp=False,  # Disable AMP for stability
    deterministic=False,  # RT-DETR doesn't support deterministic
    val=True,
    plots=True
)

### 9.3 RT-DETR Inference

In [ ]:
from ultralytics import RTDETR

# Load trained RT-DETR model
model = RTDETR('runs/rtdetr/custom_training/weights/best.pt')

# RT-DETR doesn't use NMS, so no iou parameter
results = model.predict(
    source='path/to/image.jpg',
    conf=0.25,
    save=True,
    show=True
)

# Process results
for r in results:
    boxes = r.boxes
    print(f"Detected {len(boxes)} objects (NMS-free)")
    print(f"Classes: {boxes.cls}")
    print(f"Confidences: {boxes.conf}")
    print(f"Boxes: {boxes.xyxy}")

### 9.4 RT-DETR vs YOLO Comparison

In [ ]:
from ultralytics import RTDETR, YOLO
import time

# Load models
rtdetr = RTDETR('rtdetr-l.pt')
yolo11 = YOLO('yolo11m.pt')

# Test image
test_image = 'path/to/test/image.jpg'

# RT-DETR inference
start = time.time()
rtdetr_results = rtdetr.predict(test_image, verbose=False)
rtdetr_time = time.time() - start

# YOLO inference
start = time.time()
yolo_results = yolo11.predict(test_image, verbose=False)
yolo_time = time.time() - start

print(f"RT-DETR: {len(rtdetr_results[0].boxes)} objects, {rtdetr_time*1000:.2f}ms")
print(f"YOLOv11: {len(yolo_results[0].boxes)} objects, {yolo_time*1000:.2f}ms")
print(f"\nRT-DETR advantage: No NMS required, more accurate for crowded scenes")
print(f"YOLO advantage: Faster inference, lower memory usage")

---

## 10. Comprehensive Hyperparameter Tuning Guide <a id="10-hyperparameters"></a>

### 10.1 Understanding Key Hyperparameters

#### Training Duration
- **epochs**: Number of complete passes through the dataset
- **patience**: Early stopping if no improvement for N epochs
- **time**: Maximum training time in hours

#### Batch and Image Size
- **batch**: Number of images per batch
  - Larger batch = more stable gradients but more memory
  - Use `-1` for auto batch size (60% GPU memory)
  - Use `0.7` for 70% GPU memory
- **imgsz**: Input image size (e.g., 640, 1280)
  - Larger size = better accuracy but slower training

#### Learning Rate
- **lr0**: Initial learning rate
  - SGD: typically 0.01
  - Adam/AdamW: typically 0.001
- **lrf**: Final learning rate (lr0 * lrf)
- **momentum**: Momentum factor (SGD only)
- **weight_decay**: L2 regularization

#### Warmup
- **warmup_epochs**: Gradually increase LR for stability
- **warmup_momentum**: Initial momentum during warmup
- **warmup_bias_lr**: Learning rate for bias parameters

#### Loss Weights
- **box**: Bounding box loss weight
- **cls**: Classification loss weight
- **dfl**: Distribution focal loss weight

#### Data Augmentation
- **mosaic**: Combine 4 images (0-1)
- **mixup**: Blend 2 images (0-1)
- **copy_paste**: Copy objects between images (0-1)
- **degrees**: Random rotation (-180 to +180)
- **translate**: Random translation (0-1)
- **scale**: Random scaling (>=0)
- **shear**: Random shearing (-180 to +180)
- **fliplr**: Horizontal flip probability (0-1)
- **flipud**: Vertical flip probability (0-1)
- **hsv_h/s/v**: Color jittering (0-1)

### 10.2 Hyperparameter Recipes for Different Scenarios

#### Small Dataset (< 1000 images)

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt')

# Small dataset configuration
results = model.train(
    data=dataset_path,
    epochs=200,  # More epochs for small datasets
    patience=100,
    batch=16,
    imgsz=640,
    
    # Use pretrained weights and freeze backbone
    pretrained=True,
    freeze=10,  # Freeze backbone layers
    
    # Higher learning rate since we're only training head
    lr0=0.01,
    lrf=0.01,
    
    # Strong augmentation to prevent overfitting
    mosaic=1.0,
    mixup=0.2,
    copy_paste=0.2,
    degrees=15.0,
    translate=0.2,
    scale=0.9,
    shear=5.0,
    fliplr=0.5,
    flipud=0.1,
    
    # Regularization
    weight_decay=0.001,  # Higher weight decay
    dropout=0.0,
    
    project='runs/small_dataset',
    name='training'
)

#### Large Dataset (> 10,000 images)

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11m.pt')

# Large dataset configuration
results = model.train(
    data=dataset_path,
    epochs=100,  # Fewer epochs sufficient
    patience=50,
    batch=-1,  # Auto batch size
    imgsz=640,
    
    # Can train from scratch or with less freezing
    pretrained=True,
    freeze=0,  # No freezing
    
    # Standard learning rate
    lr0=0.01,
    lrf=0.01,
    cos_lr=True,  # Cosine learning rate schedule
    
    # Moderate augmentation
    mosaic=1.0,
    mixup=0.1,
    copy_paste=0.1,
    degrees=10.0,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    
    # Multi-scale training
    multi_scale=True,
    
    # Standard regularization
    weight_decay=0.0005,
    
    # Performance optimizations
    cache=True,
    amp=True,
    workers=8,
    
    project='runs/large_dataset',
    name='training'
)

#### High Accuracy (Competition)

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11x.pt')  # Use largest model

# Competition/high-accuracy configuration
results = model.train(
    data=dataset_path,
    epochs=300,  # Long training
    patience=100,
    batch=8,  # Smaller batch for large model
    imgsz=1280,  # Large image size
    
    # Optimizer
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.0001,
    cos_lr=True,
    
    # Strong augmentation
    mosaic=1.0,
    mixup=0.2,
    copy_paste=0.3,
    degrees=15.0,
    translate=0.2,
    scale=0.9,
    shear=5.0,
    perspective=0.0005,
    fliplr=0.5,
    flipud=0.1,
    
    # Multi-scale
    multi_scale=True,
    
    # Close mosaic late
    close_mosaic=50,
    
    # Best quality
    rect=False,
    amp=True,
    
    project='runs/high_accuracy',
    name='training',
    plots=True
)

#### Fast Training (Quick Experimentation)

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt')  # Use smallest model

# Fast training configuration
results = model.train(
    data=dataset_path,
    epochs=50,  # Few epochs
    patience=20,
    batch=-1,  # Auto batch size
    imgsz=640,  # Standard size
    
    # Fast optimizer
    optimizer='SGD',
    lr0=0.01,
    lrf=0.01,
    
    # Light augmentation
    mosaic=1.0,
    mixup=0.0,
    copy_paste=0.0,
    fliplr=0.5,
    
    # Speed optimizations
    cache='ram',  # Cache in RAM
    amp=True,
    workers=8,
    rect=True,  # Rectangular training
    
    # Less frequent validation
    val=True,
    
    project='runs/fast_training',
    name='experiment'
)

### 10.3 Automatic Hyperparameter Optimization

In [ ]:
from ultralytics import YOLO

# Load model
model = YOLO('yolo11s.pt')

# Use the tune() method for automatic optimization
results = model.tune(
    data=dataset_path,
    epochs=30,  # Epochs per iteration
    iterations=300,  # Number of optimization iterations
    optimizer='AdamW',
    plots=True,
    save=True,
    val=True,
    cache=True
)

# The best hyperparameters will be saved automatically
print("Best hyperparameters found:")
print(results)

### 10.4 Manual Hyperparameter Experimentation

In [ ]:
from ultralytics import YOLO
import itertools

# Define hyperparameter grid
lr_values = [0.001, 0.01, 0.1]
batch_sizes = [8, 16, 32]
optimizers = ['SGD', 'Adam', 'AdamW']

best_map = 0
best_config = {}

# Grid search
for lr, batch, opt in itertools.product(lr_values, batch_sizes, optimizers):
    print(f"\nTesting: lr={lr}, batch={batch}, optimizer={opt}")
    
    model = YOLO('yolo11n.pt')
    
    results = model.train(
        data=dataset_path,
        epochs=30,
        lr0=lr,
        batch=batch,
        optimizer=opt,
        project='runs/grid_search',
        name=f'lr{lr}_b{batch}_{opt}',
        verbose=False
    )
    
    # Get validation metrics
    metrics = model.val()
    map50_95 = metrics.box.map
    
    print(f"mAP50-95: {map50_95:.4f}")
    
    if map50_95 > best_map:
        best_map = map50_95
        best_config = {'lr': lr, 'batch': batch, 'optimizer': opt}

print(f"\nBest configuration: {best_config}")
print(f"Best mAP50-95: {best_map:.4f}")

---

## 11. Advanced Techniques <a id="11-advanced"></a>

### 11.1 Resume Training from Checkpoint

In [ ]:
from ultralytics import YOLO

# Resume training from the last checkpoint
model = YOLO('runs/yolo11/custom_training/weights/last.pt')

# Continue training
results = model.train(
    resume=True  # Resume from last checkpoint
)

print("Training resumed and completed!")

### 11.2 Multi-GPU Training

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11m.pt')

# Train on multiple GPUs
results = model.train(
    data=dataset_path,
    epochs=100,
    batch=32,  # Total batch size across all GPUs
    device=[0, 1],  # Use GPU 0 and GPU 1
    # Or use device='0,1,2,3' for 4 GPUs
    workers=16,  # More workers for multi-GPU
    project='runs/multi_gpu',
    name='training'
)

print("Multi-GPU training completed!")

### 11.3 Training with Class Subset

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11s.pt')

# Train only on specific classes
results = model.train(
    data=dataset_path,
    epochs=100,
    classes=[0, 2, 5],  # Only train on classes 0, 2, and 5
    project='runs/class_subset',
    name='training'
)

print("Training on class subset completed!")

### 11.4 Fractional Dataset Training

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt')

# Train on only 20% of the dataset (for quick experiments)
results = model.train(
    data=dataset_path,
    epochs=50,
    fraction=0.2,  # Use only 20% of the data
    project='runs/fraction',
    name='training'
)

print("Fractional training completed!")

### 11.5 Callback Functions for Custom Behavior

In [ ]:
from ultralytics import YOLO

def on_train_epoch_end(trainer):
    """Called at the end of each training epoch."""
    print(f"Epoch {trainer.epoch} completed!")
    print(f"Loss: {trainer.loss}")

def on_val_end(validator):
    """Called at the end of validation."""
    print(f"Validation mAP50: {validator.metrics.box.map50}")

# Create model
model = YOLO('yolo11n.pt')

# Add callbacks
model.add_callback('on_train_epoch_end', on_train_epoch_end)
model.add_callback('on_val_end', on_val_end)

# Train with callbacks
results = model.train(
    data=dataset_path,
    epochs=50,
    project='runs/callbacks',
    name='training'
)

### 11.6 Model Ensemble for Better Accuracy

In [ ]:
from ultralytics import YOLO
import torch

# Load multiple trained models
models = [
    YOLO('runs/yolo11/exp1/weights/best.pt'),
    YOLO('runs/yolo11/exp2/weights/best.pt'),
    YOLO('runs/yolo11/exp3/weights/best.pt')
]

# Run inference with all models
test_image = 'path/to/test/image.jpg'
all_results = []

for i, model in enumerate(models):
    results = model.predict(test_image, verbose=False)
    all_results.append(results[0])
    print(f"Model {i+1}: {len(results[0].boxes)} detections")

# You can implement custom NMS or weighted averaging here
# For simplicity, we'll use the model with highest confidence
best_result = max(all_results, key=lambda x: x.boxes.conf.max() if len(x.boxes) > 0 else 0)

print(f"\nEnsemble result: {len(best_result.boxes)} objects")
best_result.show()

---

## 12. Model Comparison and Best Practices <a id="12-comparison"></a>

### 12.1 Model Performance Comparison

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO, RTDETR
import time

# Models to compare
models_to_test = {
    'YOLOv5s': 'yolov5s.pt',
    'YOLOv8n': 'yolov8n.pt',
    'YOLOv9c': 'yolov9c.pt',
    'YOLOv10n': 'yolov10n.pt',
    'YOLOv11n': 'yolo11n.pt',
    'RT-DETR-l': 'rtdetr-l.pt'
}

results_data = []
test_image = 'path/to/test/image.jpg'

for name, weight in models_to_test.items():
    print(f"\nTesting {name}...")
    
    # Load model
    if 'rtdetr' in weight:
        model = RTDETR(weight)
    else:
        model = YOLO(weight)
    
    # Warmup
    for _ in range(3):
        _ = model.predict(test_image, verbose=False)
    
    # Measure inference time
    times = []
    for _ in range(10):
        start = time.time()
        results = model.predict(test_image, verbose=False)
        times.append((time.time() - start) * 1000)  # Convert to ms
    
    avg_time = sum(times) / len(times)
    num_detections = len(results[0].boxes)
    
    results_data.append({
        'Model': name,
        'Inference Time (ms)': avg_time,
        'Detections': num_detections
    })
    
    print(f"{name}: {avg_time:.2f}ms, {num_detections} objects")

# Create comparison dataframe
df = pd.DataFrame(results_data)
print("\n" + "="*50)
print(df.to_string(index=False))

# Plot comparison
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(df['Model'], df['Inference Time (ms)'])
ax.set_xlabel('Model')
ax.set_ylabel('Inference Time (ms)')
ax.set_title('Model Inference Speed Comparison')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 12.2 Best Practices Summary

#### Choosing the Right Model

| Use Case | Recommended Model | Why |
|----------|------------------|-----|
| Real-time on edge devices | YOLOv11n, YOLOv10n | Smallest, fastest |
| High accuracy needed | YOLOv11x, RT-DETR-x | Best performance |
| Balanced speed/accuracy | YOLOv11s, YOLOv11m | Good tradeoff |
| Crowded scenes | RT-DETR | NMS-free, handles overlaps better |
| Stable/production | YOLOv5, YOLOv8 | Mature, well-tested |
| Latest features | YOLOv11 | Most recent improvements |

#### Training Tips

1. **Start Small**: Begin with nano/small models for quick experimentation
2. **Use Pretrained Weights**: Always start with pretrained models
3. **Freeze Layers**: For small datasets, freeze backbone layers
4. **Data Augmentation**: Use strong augmentation for small datasets
5. **Batch Size**: Use auto batch size (`batch=-1`) if unsure
6. **Learning Rate**: Start with defaults, adjust if needed
7. **Early Stopping**: Use patience to avoid overfitting
8. **Validation**: Always validate during training
9. **Cache Data**: Cache images in RAM for faster training
10. **AMP**: Enable AMP for faster training (except RT-DETR)

#### Common Issues and Solutions

| Issue | Solution |
|-------|----------|
| Out of memory | Reduce batch size or image size |
| Low accuracy | Increase epochs, use larger model, more data |
| Overfitting | Add augmentation, reduce model size, more data |
| Slow training | Enable cache, use smaller model, reduce image size |
| Training diverges | Lower learning rate, check data labels |
| No improvement | Check dataset quality, try different augmentation |

#### Deployment Considerations

- **Mobile/Edge**: Use nano models, export to TFLite or CoreML
- **Server/Cloud**: Use large models, export to ONNX or TensorRT
- **Real-time Video**: Use YOLOv10/YOLOv11 with TensorRT
- **High Precision**: Use RT-DETR or YOLOv11x with large image size

### 12.3 Complete Training Pipeline Example

In [ ]:
from ultralytics import YOLO
import os

# Configuration
MODEL_NAME = 'yolo11s.pt'
DATA_YAML = 'dataset.yaml'
PROJECT_NAME = 'my_project'
EXPERIMENT_NAME = 'experiment_1'

# 1. Load model
print("[1/5] Loading model...")
model = YOLO(MODEL_NAME)

# 2. Train
print("[2/5] Starting training...")
results = model.train(
    data=DATA_YAML,
    epochs=100,
    imgsz=640,
    batch=-1,
    device=0,
    project=f'runs/{PROJECT_NAME}',
    name=EXPERIMENT_NAME,
    patience=50,
    cache=True,
    amp=True,
    plots=True
)

# 3. Validate
print("[3/5] Validating model...")
metrics = model.val()
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"mAP50: {metrics.box.map50:.4f}")

# 4. Test inference
print("[4/5] Testing inference...")
test_results = model.predict(
    source='path/to/test/images/',
    save=True,
    conf=0.25
)
print(f"Processed {len(test_results)} images")

# 5. Export
print("[5/5] Exporting model...")
model.export(format='onnx')
print(f"Model exported to ONNX format")

print("\n" + "="*50)
print("Training pipeline completed successfully!")
print(f"Best weights: runs/{PROJECT_NAME}/{EXPERIMENT_NAME}/weights/best.pt")
print("="*50)

---

## Conclusion

Congratulations! You've completed the comprehensive course on fine-tuning Ultralytics models. You now know how to:

- Train and fine-tune YOLOv5, YOLOv8, YOLOv9, YOLOv10, YOLOv11, and RT-DETR
- Run inference and evaluate model performance
- Modify and optimize hyperparameters for different scenarios
- Apply advanced techniques like multi-GPU training, callbacks, and ensembles
- Choose the right model for your specific use case

### Next Steps

1. **Experiment**: Try different models and hyperparameters on your dataset
2. **Read Documentation**: Visit [docs.ultralytics.com](https://docs.ultralytics.com/)
3. **Join Community**: Participate in Ultralytics GitHub discussions
4. **Deploy**: Export your model and deploy to production
5. **Keep Learning**: Stay updated with new YOLO versions and features

### Resources

- **Ultralytics Documentation**: https://docs.ultralytics.com/
- **GitHub Repository**: https://github.com/ultralytics/ultralytics
- **YOLOv5 GitHub**: https://github.com/ultralytics/yolov5
- **Roboflow Universe**: https://universe.roboflow.com/ (datasets)
- **Papers with Code**: https://paperswithcode.com/ (research papers)

Happy training!